**Table of contents**<a id='toc0_'></a>    
- [Libraries](#toc1_)    
- [Loading text filings](#toc2_)    
- [Extracting the Management's Discussion and Analysis Section](#toc3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Libraries](#toc0_)

In [1]:
import os
import time
os.chdir(os.environ.get('PROJECT_PATH'))
from secnlp.ml_logic import data as d
from secnlp.ml_logic import parsing as p
import secnlp.ml_logic.parsing
from secnlp import utils as u
from secnlp.params import *
import pandas as pd
import importlib

# <a id='toc2_'></a>[Loading text filings](#toc0_)

In [2]:
df = u.read_data_from_bq(credentials = SERVICE_ACCOUNT, gcp_project = PROJECT, bq_dataset = DATASET_ID, table = FILINGS_10KQ_TABLE_ID)

In [3]:
df['date_filed'] = pd.to_datetime(df['date_filed'])

In [9]:
filing_sample_10k = df[(df['date_filed'].dt.year == 2023) & (df['form_type'] == '10-K')].sample(1)
filing_sample_10k['raw_filing'] = filing_sample_10k['file_name'].apply(lambda url: d.fetch_text_from_url(url, agent = AGENT))


In [5]:
filing_sample_10q = df[(df['date_filed'].dt.year == 2023) & (df['form_type'] == '10-Q')].sample(1)
filing_sample_10q['raw_filing'] = filing_sample_10q['file_name'].apply(lambda url: d.fetch_text_from_url(url, agent = AGENT))


# 10-Ks

In [10]:
text_10k = filing_sample_10k['raw_filing'].iloc[0]

In [11]:
mda_10k = p.parse_10k_filing_items(text_10k, item = '7')

In [12]:
importlib.reload(secnlp.ml_logic.parsing)
p.cleaning(mda_10k)

'item 7 management8217s discussion and analysis of financial condition and results of operations omitted'

# 10-Q

In [20]:
text_10q = filing_sample_10q['raw_filing'].iloc[0]
mda_10q = p.parse_10q_filing_items(text_10q, item = '2')

In [21]:
importlib.reload(secnlp.ml_logic.parsing)
p.cleaning(mda_10q)

'item 2 item 2 management  s discussion and analysis of financial condition and results of operations readers are cautioned that certain statements contained herein are forwardlooking statements and should be read in conjunction with our disclosures under the heading forwardlooking statements above these statements are based on current expectations and assumptions that are subject to risks and uncertainties this discussion also should be read in conjunction with the notes to our consolidated financial statements contained in item 8 financial statements and supplementary data of this report operations overviewoutlook the company developed a document called the creds deck which provides a description to prospective clients of digital claritys value proposition the fiscal year 2023 has focused on a slow return to normalcy though businesses have faced enormous challenges over the past few years and dbmms operating business digital clarity is no exception however for context it is worth rem

In [22]:
filing_sample_10q

,cik,company,form_type,date_filed,file_name,raw_filing
878217,1127475,"Digital Brand Media & Marketing Group, Inc.",10-Q,2023-04-13,edgar/data/1127475/0001185185-23-000330.txt,<SEC-DOCUMENT>0001185185-23-000330.txt : 20230...
